# 다이아몬드 가격 예측 모델

PyTorch를 사용한 다이아몬드 가격 예측 신경망 모델입니다.

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

## 신경망 모델 클래스 정의

In [ ]:
class DiamondPricePredictor(nn.Module):
    """다이아몬드 가격 예측을 위한 신경망 모델"""
    
    def __init__(self, input_features):
        super(DiamondPricePredictor, self).__init__()
        self.fc1 = nn.Linear(input_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.output(x)
        return x

## 메인 모델 클래스 정의 - Part 1: 초기화 및 데이터 전처리

In [ ]:
class DiamondPriceModel:
    """다이아몬드 가격 예측 모델 클래스"""
    
    def __init__(self, data_path, test_size=0.2, batch_size=32, learning_rate=0.001):
        self.data_path = data_path
        self.test_size = test_size
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.scaler = StandardScaler()
        self.model = None
        self.criterion = nn.MSELoss()
        self.optimizer = None
        
    def load_and_preprocess_data(self):
        """데이터 로드 및 전처리"""
        # 데이터 로드
        diamond = pd.read_csv(self.data_path)
        print("데이터 기본 정보:")
        print(diamond.head())
        print(diamond.info())
        print(diamond.describe())
        
        # 특성과 타겟 분리 (price 컬럼 제외한 모든 컬럼을 특성으로 사용)
        X = diamond.drop(['price'], axis=1)
        y = diamond['price']
        
        print(f"\n원본 특성 수: {X.shape[1]}")
        print(f"데이터 포인트 수: {X.shape[0]}")
        
        # 원핫 인코딩 (범주형 변수 처리)
        X = pd.get_dummies(X, drop_first=True)
        print(f"원핫 인코딩 후 특성 수: {X.shape[1]}")
        
        # 데이터 표준화
        X_scaled = self.scaler.fit_transform(X)
        
        # 훈련/테스트 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=self.test_size, random_state=42
        )
        
        # 텐서로 변환
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)
        
        # 데이터로더 생성
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        
        self.train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        self.test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
        
        return X.shape[1]  # 특성 수 반환

## 메인 모델 클래스 정의 - Part 2: 모델 생성 및 훈련

In [ ]:
    def build_model(self, input_features):
        """모델 생성 및 최적화기 설정"""
        self.model = DiamondPricePredictor(input_features)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        print(f"모델 생성 완료 - 입력 특성 수: {input_features}")
    
    def train(self, epochs=100):
        """모델 훈련"""
        if self.model is None:
            raise ValueError("모델이 생성되지 않았습니다. build_model()을 먼저 호출하세요.")
        
        print(f"\n모델 훈련 시작 - {epochs} 에포크")
        self.model.train()
        
        for epoch in range(epochs):
            epoch_loss = 0.0
            num_batches = 0
            
            for inputs, labels in self.train_loader:
                # 그래디언트 초기화
                self.optimizer.zero_grad()
                
                # 순전파
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                # 역전파 및 최적화
                loss.backward()
                self.optimizer.step()
                
                epoch_loss += loss.item()
                num_batches += 1
            
            avg_loss = epoch_loss / num_batches
            if (epoch + 1) % 10 == 0:  # 10 에포크마다 출력
                print(f'Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}')
        
        print("훈련 완료!")

## 메인 모델 클래스 정의 - Part 3: 평가 및 예측

In [ ]:
    def evaluate(self):
        """모델 평가"""
        if self.model is None:
            raise ValueError("모델이 생성되지 않았습니다.")
        
        print("\n모델 평가 시작...")
        self.model.eval()
        
        total_loss = 0.0
        total_samples = 0
        predictions = []
        actuals = []
        
        with torch.no_grad():
            for inputs, labels in self.test_loader:
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                total_loss += loss.item() * inputs.size(0)
                total_samples += inputs.size(0)
                
                predictions.extend(outputs.squeeze().tolist())
                actuals.extend(labels.squeeze().tolist())
        
        # 평가 지표 계산
        avg_mse = total_loss / total_samples
        rmse = np.sqrt(avg_mse)
        
        # 추가 평가 지표 계산
        predictions = np.array(predictions)
        actuals = np.array(actuals)
        mae = np.mean(np.abs(predictions - actuals))
        
        print(f"평가 결과:")
        print(f"  평균 MSE: {avg_mse:.4f}")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAE: {mae:.4f}")
        
        return avg_mse, rmse, mae
    
    def predict(self, X_new):
        """새로운 데이터에 대한 예측"""
        if self.model is None:
            raise ValueError("모델이 생성되지 않았습니다.")
        
        self.model.eval()
        X_scaled = self.scaler.transform(X_new)
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        
        with torch.no_grad():
            predictions = self.model(X_tensor)
        
        return predictions.squeeze().numpy()

## 모델 인스턴스 생성 및 설정

In [ ]:
# 모델 인스턴스 생성
model = DiamondPriceModel(
    data_path="./data/diamonds.csv",  # 데이터 파일 경로를 실제 경로로 수정하세요
    test_size=0.2,
    batch_size=32,
    learning_rate=0.001
)

print("모델 인스턴스 생성 완료!")

## 데이터 전처리 실행

In [ ]:
# 데이터 전처리
try:
    input_features = model.load_and_preprocess_data()
    print(f"\n데이터 전처리 완료! 특성 수: {input_features}")
except FileNotFoundError:
    print("데이터 파일을 찾을 수 없습니다. 경로를 확인하세요.")
except Exception as e:
    print(f"데이터 전처리 중 오류 발생: {e}")

## 모델 생성

In [ ]:
# 모델 생성
model.build_model(input_features)

## 모델 훈련

In [ ]:
# 모델 훈련 (에포크 수를 조정할 수 있습니다)
model.train(epochs=100)

## 모델 평가

In [ ]:
# 모델 평가
mse, rmse, mae = model.evaluate()

# 결과 시각화를 위한 추가 분석
print(f"\n=== 최종 평가 메트릭 ===")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

## 예측 예제 (선택사항)

새로운 데이터에 대한 예측을 위한 예제 코드입니다.

In [ ]:
# 새로운 데이터에 대한 예측 예제
# 주의: 실제 사용 시에는 원본 데이터와 동일한 형태의 데이터를 준비해야 합니다.
# 예시 코드이므로 실제 데이터 구조에 맞게 수정이 필요합니다.

print("=== 예측 사용법 안내 ===")
print("다음과 같은 형태로 새로운 데이터를 준비하여 예측할 수 있습니다:")
print()
print("예제 코드:")
print("new_data = pd.DataFrame({")
print("    'carat': [1.0, 2.0],")
print("    'cut': ['Premium', 'Ideal'],")
print("    'color': ['G', 'H'],")
print("    'clarity': ['SI1', 'VS2'],")
print("    # ... 기타 필요한 특성들")
print("})")
print()
print("# 원핫 인코딩 적용 (훈련 데이터와 동일하게)")
print("new_data_encoded = pd.get_dummies(new_data, drop_first=True)")
print()
print("# 예측 실행")
print("predictions = model.predict(new_data_encoded)")
print("print(f'예측된 다이아몬드 가격: {predictions}')")